In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Import my Kovaak's data loader
from kovaaks_loader import load_all_kovaaks_data

print(os.getcwd())
csv_dir = "./stats"


/Users/aidansmith/Library/CloudStorage/OneDrive-Personal/DocumentsGrad/Projects/AimTrainerCompanion/KovaaKs-Data-Analysis-Companion


In [2]:
# Load all data
main_data, weapon_data, stats_data = load_all_kovaaks_data(csv_dir)

Processed 10 files...
Processed 20 files...
Processed 30 files...
Processed 40 files...
Processed 50 files...
Processed 60 files...
Processed 70 files...
Processed 80 files...
Processed 90 files...
Processed 100 files...
Processed 110 files...
Processed 120 files...
Processed 130 files...
Processed 140 files...
Processed 150 files...
Processed 160 files...
Processed 170 files...
Processed 180 files...
Processed 190 files...
Processed 200 files...
Processed 210 files...
Processed 220 files...
Processed 230 files...
Processed 240 files...
Processed 250 files...
Processed 260 files...
Processed 270 files...
Processed 280 files...
Processed 290 files...
Processed 300 files...
Processed 310 files...
Processed 320 files...
Processed 330 files...
Processed 340 files...

Total files processed: 343
Unique tasks: 102


# Analyzing Sensitivity

Need to check:
- How many different sensitivities are used (differences in DPI and Sens)
- How inner-task performance difference changes between task type

Should probably primarily focus on eDPI (so dpi * sensitivity)
Maybe at most one general analysis of performance to potentially catch performance losses from low DPI (low dpi -> larger distance before single movement chunk -> potentially worse tracking, smoothness, etc)